<a href="https://colab.research.google.com/github/na0495/spark-mini-project/blob/master/spark_solutiuon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 39.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=dbb65996fd90a1137e4ae1414d8683262731a3051511fa82f89472c7a2291423
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


1. Définir les DF associées à chaque donnée.

In [28]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# read df from json file
orders_df = spark.read.option("multiline", "true").json("data/orders.json")
addresses_df = spark.read.option("multiline", "true").json("data/addresses.json")
prices_df = spark.read.option("multiline", "true").json("data/prices.json")
products_df = spark.read.option("multiline", "true").json("data/products.json")

2. Créer une vue pour chaque DF createOrReplaceTempView

In [38]:
# createOrReplaceTempView for each df
orders_df.createOrReplaceTempView("orders")
addresses_df.createOrReplaceTempView("addresses")
prices_df.createOrReplaceTempView("prices")
products_df.createOrReplaceTempView("products")

spark.sql("SELECT * FROM orders").show()
spark.sql("SELECT * FROM addresses").show()
spark.sql("SELECT * FROM prices").show()
spark.sql("SELECT * FROM products").show()


+------+-----+-------+------+--------+
|agence|annee|article|client|quantite|
+------+-----+-------+------+--------+
|    75| 2021|      3|  toto|       2|
|    75| 2020|      2|  toto|       1|
|    51| 2021|      1|  toto|       5|
|    51| 2021|      2|  fofo|       6|
+------+-----+-------+------+--------+

+------+---+
|adress| id|
+------+---+
| paris| 75|
| reims| 51|
|  nice|  6|
+------+---+

+---+----+
|idm|prix|
+---+----+
|  1|  10|
|  2|  20|
|  3|  70|
+---+----+

+---+-------+-------+
|idm| marque|produit|
+---+-------+-------+
|  1|   sony| souris|
|  2|samsung|clavier|
|  3|  apple|  ecran|
+---+-------+-------+



3. Proposer des réponses pour les requêtes suivantes :
*   Déterminer les prix de chaque produit







In [32]:
# get the price of each product
products_df.select("idm", 'produit').join(prices_df, "idm").show()


+---+-------+----+
|idm|produit|prix|
+---+-------+----+
|  1| souris|  10|
|  2|clavier|  20|
|  3|  ecran|  70|
+---+-------+----+



* Déterminer les articles que toto a acheté en 2021

In [42]:
orders_df.select("annee", "article").filter(orders_df.annee.like("%2021%")).show()

+-----+-------+
|annee|article|
+-----+-------+
| 2021|      3|
| 2021|      1|
| 2021|      2|
+-----+-------+



* Montant dépensé par toto par agence

In [64]:
orders_df.join(prices_df, orders_df.article == prices_df.idm, "left").filter(orders_df.client.like("toto")).groupBy("agence").sum("prix").show()


+------+---------+
|agence|sum(prix)|
+------+---------+
|    51|       10|
|    75|       90|
+------+---------+



* Déterminer le CA pour chaque agence en 2021

In [94]:
turnover_df = orders_df.join(prices_df, orders_df.article == prices_df.idm, "left")
tr = turnover_df.withColumn("turnover", turnover_df["prix"] * turnover_df["quantite"]).select("agence", "turnover").orderBy("agence")
tr.groupBy("agence").sum("turnover").show()

+------+-------------+
|agence|sum(turnover)|
+------+-------------+
|    51|          170|
|    75|          160|
+------+-------------+



* Quel est le produit qui a le plus été acheté en 2021 ?


In [95]:
orders_df.join(prices_df, orders_df.article == prices_df.idm, "left").groupBy("article").sum("quantite").orderBy("sum(quantite)", ascending=False).show()

+-------+-------------+
|article|sum(quantite)|
+-------+-------------+
|      2|            7|
|      1|            5|
|      3|            2|
+-------+-------------+



* Quel est le client qui a acheté le plus d’article ?

In [96]:
orders_df.join(prices_df, orders_df.article == prices_df.idm, "left").groupBy("client").sum("quantite").orderBy("sum(quantite)", ascending=False).show()

+------+-------------+
|client|sum(quantite)|
+------+-------------+
|  toto|            8|
|  fofo|            6|
+------+-------------+



* Qui est le client qui a dépensé le plus d’argent ?

In [97]:
orders_df.join(prices_df, orders_df.article == prices_df.idm, "left").groupBy("client").sum("prix").orderBy("sum(prix)", ascending=False).show()

+------+---------+
|client|sum(prix)|
+------+---------+
|  toto|      100|
|  fofo|       20|
+------+---------+

